# Check PSF correction in A360 field

Authors: Céline Combet, Andrés Plazas Malagón (with inputs from many)\
LSST Science Piplines version: Weekly 2025_17\
Container Size: medium

This notebook provide the code used to generate the figures of [SITCOMTN-161](https://sitcomtn-161.lsst.io/) (it acutally also produces other figures that were not shown in the TechNote), which aims at checking the PSF behaviour and correction in A360 field. The main steps are

- Loading the relevant stars from the object catalogs (all tracts and patches needed) using the butler
- Checking out the size of the PSF accross the field
- Computing the ellipticities of stars and corresponding PSF model and make the whisker plots to check the residuals.
- Computing the tangential shear of the residuals
- Exploring the rho-statistics of the residuals

NB: Check out the [PSF DP1 tutorial](\url{https://dp1.lsst.io/tutorials/notebook/304/notebook-304-1.html}) for more PSF diagnostics 

In [ ]:
# general python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from lsst.daf.butler import Butler
import lsst.geom as geom
import lsst.afw.geom as afwGeom

In [ ]:
repo = '/repo/dp1'
collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'

butler = Butler(repo, collections=collection)
skymap = butler.get('skyMap', skymap='lsst_cells_v1')

In [ ]:
version_str = collection.split('/')
version = version_str[-2:][0]+'_'+version_str[-2:][1]
version

## Band under scrutiny

In [ ]:
my_band = 'i'

## Load the relevant catalogs
For PSF studies, we need to look at stars
### Find all tracts/patches to load

In [ ]:
# Position of the BCG for A360
ra_bcg = 37.862
dec_bcg = 6.98

# Looking for all patches in delta deg region around it
delta = 0.5
center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
         geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

tracts_and_patches = skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
#tp_dict

### Load quantities with the cuts needed to get PSF stars, etc. 

In [ ]:
# Get the object catlaog of these patches
if 'v29' in version:
    datasetType = 'object_patch'
else:
    datasetType = 'objectTable'

merged_cat_used = pd.DataFrame() # to store the catalog of stars used by PIFF for the PSF modeling
merged_cat_reserved = pd.DataFrame() # to store the catalog of stars marked as "reserved", i.e. not used to build the PIFF PSF model 
merged_cat_all = pd.DataFrame() # to store all extended objects, to have more locations to check the PSF model.

for tract in list(tp_dict.keys()):
    print(f'Loading objects from tract {tract}, patches:{tp_dict[tract]}')

    for patch in tp_dict[tract]:
        dataId = {'tract': tract, 'patch' : patch ,'skymap':'lsst_cells_v1'}
        obj_cat = butler.get(datasetType, dataId=dataId)
        if datasetType == 'object_patch': # new naming convention, and obj_cat is now an astropy table. 
            obj_cat = obj_cat.to_pandas() # convert to pandas to leave the rest of the code unchanged

        # Stars used for the PSF modeling
        filt1 = obj_cat['detect_isPrimary'] == True
        filt1 &= obj_cat['refExtendedness'] == 0.0 # keep stars only
        filt1 &= obj_cat[f'{my_band}_calib_psf_used'] == True # that were used to build the psf model
        filt1 &= obj_cat[f'{my_band}_pixelFlags_inexact_psfCenter'] == False # To avoid objects with discontinuous PSF (due to edges)
        merged_cat_used = pd.concat([merged_cat_used, obj_cat[filt1]], ignore_index=True)
        
        # Stars "reserved" to check the PSF modeling
        filt2 = obj_cat['detect_isPrimary'] == True
        filt2 &= obj_cat['refExtendedness'] == 0.0
        filt2 &= obj_cat[f'{my_band}_pixelFlags_inexact_psfCenter']==False
        filt2 &= obj_cat[f'{my_band}_calib_psf_reserved'] == True # not used for the psf model

        merged_cat_reserved = pd.concat([merged_cat_reserved, obj_cat[filt2]], ignore_index=True)

        # All extended objects (to have more locations where to look at the PSF size and ellipticity)
        filt3 = obj_cat['detect_isPrimary']==True
        filt3 &= obj_cat['refExtendedness'] == 1.0
        merged_cat_all = pd.concat([merged_cat_all, obj_cat[filt3]], ignore_index=True)

In [ ]:
print(f'used stars: {len(merged_cat_used)}; reserved stars:{len(merged_cat_reserved)}, extended objects: {len(merged_cat_all)}')

## Check out the location of the PSF stars, in (ra, dec) and (x,y) coordinates, colored by track number

The BCG and 1 deg field around it are highlighted in the (ra,dec) plot

In [ ]:
from matplotlib.patches import Circle

circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='black', fill=False, linewidth=0.5)

color = ['red', 'blue','green','magenta']

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
for i,tract in enumerate(list(tp_dict.keys())):
    filt = merged_cat_used['tract'] == tract
    ax[0].scatter(merged_cat_used[filt]['coord_ra'], merged_cat_used[filt]['coord_dec'], 
                  c=color[i],  marker='.', s=2, label=f'tract = {tract}')    
    ax[0].set_xlabel('ra [deg]')
    ax[0].set_ylabel('dec [deg]')
    ax[0].add_patch(circle1)

for i,tract in enumerate(list(tp_dict.keys())):
    filt = merged_cat_used['tract'] == tract
    ax[1].scatter(merged_cat_used[filt]['i_centroid_x'], merged_cat_used[filt]['i_centroid_y'],  marker='.', s=2, c=color[i])
ax[1].set_xlabel('i_centroid_x')
ax[1].set_ylabel('i_centroid_y')
fig.tight_layout()

ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='black')
ax[0].invert_xaxis() # to have ra increase to the left/east

fig.legend(loc=9, markerscale=10)

In (ra,dec), the stars cover the field and we can see which tract contribute to which area. In (x,y), we see a clear gap between the tracts as each tract has its own x, y coordinate system, (and some of these tracts do not have any visits covering some parts of them). Nontheless, looking at the patterns in the tracts, we see that the (x,y) grid is align with (ra,dec).

## PSF size variation across the field

The `{band}_i{xx,xy,yy}PSF` quantities are the second moment of the PSF model for each object location in the catalog. The trace radius (PSF size) of the PSF is defined as
$r_t = \sqrt{(I_{xx} + I_{yy}/2)}$

We look at the size of the PSF:
- at the location of `used` stars (`merged_cat_used` catalog)
- at the location of all extended objects (`merged_cat_all` catalog), to have a better coverage of the field and visualize PSF discontinutities, etc.


In [ ]:
size = np.sqrt((merged_cat_used[f'{my_band}_ixxPSF'] + merged_cat_used[f'{my_band}_iyyPSF']) / 2)
size_all = np.sqrt((merged_cat_all[f'{my_band}_ixxPSF'] + merged_cat_all[f'{my_band}_iyyPSF']) / 2) # at all extended objects locations (not only PSF stars)

In [ ]:
from matplotlib.patches import Circle

ra, dec =  merged_cat_used['coord_ra'], merged_cat_used['coord_dec']

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,5))

scatter_plot1 = ax[0].scatter(ra, dec, c=size, s=4, cmap='viridis', marker='o')
circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='orange', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')

ax[0].set_xlabel('ra [deg]')
ax[0].set_ylabel('dec [deg]')
ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
ax[0].invert_xaxis()
ax[0].add_patch(circle1)
#ax.add_patch(circle2)

scatter_plot2 = ax[1].scatter(merged_cat_all['coord_ra'], merged_cat_all['coord_dec'], 
                              c=size_all, s=1, cmap='viridis', marker='o')
ax[1].set_xlabel('ra [deg]')
ax[1].set_ylabel('dec [deg]')
ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
circle2 = Circle((ra_bcg, dec_bcg), 0.5, color='orange', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
ax[1].invert_xaxis()
ax[1].add_patch(circle2)

plt.colorbar(scatter_plot1, ax=ax[0], label='PSF size [pixels]')
plt.colorbar(scatter_plot1, ax=ax[1], label='PSF size [pixels]')

The PSF size is varying by ~0.5 pixel across A360 field. The figure on the right showis the PSF size at more 
locations, which highlights the discontinuities in the PSF modeling when close to edges. Also allows us to see the various orientation of visits used to build the coadd.

## Whisker plots - PSF ellipticity and PSF correction.

The ellipticity components $e_1$, $e_2$ are computed from moments as:

$e_1 = (I_{xx} - I_{yy}) / (I_{xx} + I_{yy})$

$e_2 = 2I_{xy} / (I_{xx} + I_{yy})$

The from this, the amplitude and orientation of the ellipse (angle of the ellipse major axis with respect to the (x,y) coordinate frame) are given by

$e = \sqrt{e_1^2 + e_2^2}$+

$\theta = 0.5 \times \arctan (e_2/e_1)$

We also define the trace of the second moments matrix:

$T = I_{xx} + I_{yy}$

In [ ]:
def get_psf_ellip(catalog, band=my_band):
    psf_mxx = catalog[f'{band}_ixxPSF']
    psf_myy = catalog[f'{band}_iyyPSF']
    psf_mxy = catalog[f'{band}_ixyPSF']
    return (psf_mxx - psf_myy) / (psf_mxx + psf_myy), 2.* psf_mxy / (psf_mxx + psf_myy)


def get_star_ellip(catalog, band=my_band):
    star_mxx = catalog[f'{band}_ixx']
    star_myy = catalog[f'{band}_iyy']
    star_mxy = catalog[f'{band}_ixy']
    return (star_mxx - star_myy) / (star_mxx + star_myy), 2. * star_mxy / (star_mxx + star_myy)

def get_psf_T(catalog, band=my_band):
    return catalog[f'{band}_ixxPSF'] + catalog[f'{band}_iyyPSF']

def get_star_T(catalog, band=my_band):
    return catalog[f'{band}_ixx'] + catalog[f'{band}_iyy']


In [ ]:
# For the PSF model, at the location of `used` stars

T_psf_used = get_psf_T(merged_cat_used)

e1_psf_used, e2_psf_used = get_psf_ellip(merged_cat_used)
e_psf_used = np.sqrt(e1_psf_used*e1_psf_used + e2_psf_used*e2_psf_used) # module of ellipticity
theta_psf_used = 0.5 * np.arctan2(e2_psf_used,e1_psf_used)

cx_psf_used = e_psf_used * np.cos(theta_psf_used) # x-component of the vector for the whisker plot
cy_psf_used = e_psf_used * np.sin(theta_psf_used) # y-component of the vector for the whisker plot

# For the PSF model, at the location of `reserved` stars
T_psf_reserved = get_psf_T(merged_cat_reserved)

e1_psf_reserved, e2_psf_reserved = get_psf_ellip(merged_cat_reserved)
e_psf_reserved = np.sqrt(e1_psf_reserved*e1_psf_reserved + e2_psf_reserved*e2_psf_reserved) # module of ellipticity
theta_psf_reserved = 0.5 * np.arctan2(e2_psf_reserved,e1_psf_reserved)

cx_psf_reserved = e_psf_reserved * np.cos(theta_psf_reserved) # x-component of the vector for the whisker plot
cy_psf_reserved = e_psf_reserved * np.sin(theta_psf_reserved) # y-component of the vector for the whisker plot

e1_psf_all, e2_psf_all = get_psf_ellip(merged_cat_all)

In [ ]:
# Repeat for the `used` stars
T_star_used = get_star_T(merged_cat_used)

e1_star_used, e2_star_used = get_star_ellip(merged_cat_used)
e_star_used = np.sqrt(e1_star_used*e1_star_used+e2_star_used*e2_star_used)
theta_star_used = 0.5 * np.arctan2(e2_star_used,e1_star_used)

cx_star_used = e_star_used * np.cos(theta_star_used)
cy_star_used = e_star_used * np.sin(theta_star_used)

# Repeat for the residual of the 'used' stars
e1_residual_used = e1_star_used - e1_psf_used
e2_residual_used = e2_star_used - e2_psf_used
theta_residual_used = 0.5 * np.arctan2(e2_residual_used,e1_residual_used)
e_residual_used = np.sqrt(e1_residual_used*e1_residual_used + e2_residual_used*e2_residual_used)

cx_residual_used = e_residual_used * np.cos(theta_residual_used)
cy_residual_used = e_residual_used * np.sin(theta_residual_used)

# Repeat for the `reserved` stars
T_star_reserved = get_star_T(merged_cat_reserved)

e1_star_reserved, e2_star_reserved = get_star_ellip(merged_cat_reserved)
e_star_reserved = np.sqrt(e1_star_reserved*e1_star_reserved+e2_star_reserved*e2_star_reserved)
theta_star_reserved = 0.5 * np.arctan2(e2_star_reserved,e1_star_reserved)

cx_star_reserved = e_star_reserved * np.cos(theta_star_reserved)
cy_star_reserved = e_star_reserved * np.sin(theta_star_reserved)

# Repeat for the residual of the 'reserved' stars
e1_residual_reserved = e1_star_reserved - e1_psf_reserved
e2_residual_reserved = e2_star_reserved - e2_psf_reserved
theta_residual_reserved = 0.5 * np.arctan2(e2_residual_reserved,e1_residual_reserved)
e_residual_reserved = np.sqrt(e1_residual_reserved*e1_residual_reserved + e2_residual_reserved*e2_residual_reserved)

cx_residual_reserved = e_residual_reserved * np.cos(theta_residual_reserved)
cy_residual_reserved = e_residual_reserved * np.sin(theta_residual_reserved)



### for PSF stars, in (x,y) coordinates

In [ ]:
x_centroid, y_centroid =  merged_cat_used[f'{my_band}_centroid_x'], merged_cat_used[f'{my_band}_centroid_y']

scale = 1.e-4

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
ax[0].quiver(x_centroid, y_centroid, cx_star_used, cy_star_used, angles='xy', color='black',
           scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
         label='Star ellipticity')
ax[0].set_title(f'Star ellipticity (from {my_band}_ixx, {my_band}_ixy, {my_band}_iyy)')
ax[0].set_xlabel('x_centroid')
ax[0].set_ylabel('y_centroid')

ax[1].quiver(x_centroid, y_centroid, cx_psf_used, cy_psf_used, angles='xy', color='black',
           scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
         label='PSF model ellipticity')
ax[1].set_xlabel('x_centroid')
ax[1].set_title(f'PSF model ellipticity (from {my_band}_ixxPSF, {my_band}_ixyPSF, {my_band}_iyyPSF)')

ax[2].quiver(x_centroid, y_centroid, cx_residual_used, cy_residual_used, angles='xy', color='black',
            scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0)
ax[2].set_xlabel('x_centroid')
ax[2].set_title('Star - PSF ellipticity residuals')

ref_norm = 0.1  # Define reference vector norm
ref_x, ref_y = 3000, 5000  # Position to place reference vector
ax[0].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[1].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[2].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')

ax[0].text(ref_x + ref_norm / (2*scale), ref_y + 500, f'e=0.1', color='red', ha='center')
ax[1].text(ref_x + ref_norm / (2*scale), ref_y + 500, f'e=0.1', color='red', ha='center')
ax[2].text(ref_x + ref_norm / (2*scale), ref_y + 500, f'e=0.1', color='red', ha='center')

# 
fig.tight_layout()

### for PSF stars in (ra, dec)

To be consistent with ra increasing to the left, need to switch the sign of the `cx` component for the plots.

In [ ]:
x_centroid, y_centroid =  merged_cat_used['coord_ra'], merged_cat_used['coord_dec']

scale = 2.2

circle0 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle2 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')


fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

ax[0].quiver(x_centroid, y_centroid, -cx_star_used, cy_star_used, angles='xy', color='black',
           scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
         label='Star ellipticity', pivot='middle')
ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[0].add_patch(circle0)
ax[0].invert_xaxis()
ax[0].set_title(f'Star ellipticity (from {my_band}_ixx, {my_band}_ixy, {my_band}_iyy)')
ax[0].set_xlabel('ra [deg]')
ax[0].set_ylabel('dec [deg]')
ax[0].set_xlim([38.6, 37.2])
ax[0].set_ylim([6.23, 7.6])

ax[1].quiver(x_centroid, y_centroid, -cx_psf_used, cy_psf_used, angles='xy', color='black',
           scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
         label='PSF model ellipticity', pivot='middle')
ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[1].add_patch(circle1)
ax[1].invert_xaxis()
ax[1].set_xlabel('ra [deg]')
ax[1].set_title(f'PSF model ellipticity (from {my_band}_ixxPSF, {my_band}_ixyPSF, {my_band}_iyyPSF)')
ax[1].set_xlim([38.6, 37.2])
ax[1].set_ylim([6.23, 7.6])

ax[2].quiver(x_centroid, y_centroid, -cx_residual_used, cy_residual_used, angles='xy', color='black',
            scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0, pivot='middle')
ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[2].add_patch(circle2)
ax[2].invert_xaxis()
ax[2].set_xlabel('ra [deg]')
ax[2].set_title('Star - PSF ellipticity residuals')
ax[2].set_xlim([38.6, 37.2])
ax[2].set_ylim([6.23, 7.6])

ref_norm = 0.1  # Define reference vector norm
ref_x, ref_y = 38.4, 6.25  # Position to place reference vector
ax[0].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[1].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[2].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')

ax[0].text(ref_x + ref_norm / (2*scale), ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[1].text(ref_x + ref_norm / (2*scale), ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[2].text(ref_x + ref_norm / (2*scale), ref_y + 0.02, f'e=0.1', color='red', ha='center')


# ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[1].add_patch(circle1)
# ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[2].add_patch(circle1)
# 
fig.tight_layout()

The violet circle is a 0.5 deg field around the BCG.

### for `reserved` stars (that haven't been used by PIFF), in (ra, dec)

Now we repeat the same thing with the `reserved` stars, that were not used to buid the PSF model. NB: there are far less reserved stars than PSF stars.

In [ ]:
x_centroid, y_centroid =  merged_cat_reserved['coord_ra'], merged_cat_reserved['coord_dec']

scale = 2.2


circle0 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle1 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')
circle2 = Circle((ra_bcg, dec_bcg), 0.5, color='darkviolet', fill=False, linewidth=1, 
                label='0.5 deg field around BCG')


fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

ax[0].quiver(x_centroid, y_centroid, -cx_star_reserved, cy_star_reserved, angles='xy', color='black',
           scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
         label='Star ellipticity')
ax[0].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet', label='cluster 0.5 deg field')
ax[0].add_patch(circle0)
ax[0].invert_xaxis()
ax[0].set_title(f'Star ellipticity (from {my_band}_ixx, {my_band}_ixy, {my_band}_iyy)')
ax[0].set_xlabel('ra [deg]')
ax[0].set_ylabel('dec [deg]')
ax[0].set_xlim([38.6, 37.2])
ax[0].set_ylim([6.25, 7.6])

ax[1].quiver(x_centroid, y_centroid, -cx_psf_reserved, cy_psf_reserved, angles='xy', color='black',
           scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
         label='PSF model ellipticity')
ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[1].add_patch(circle1)
ax[1].invert_xaxis()
ax[1].set_xlabel('ra [deg]')
ax[1].set_title(f'PSF model ellipticity (from {my_band}_ixxPSF, {my_band}_ixyPSF, {my_band}_iyyPSF)')
ax[1].set_xlim([38.6, 37.2])
ax[1].set_ylim([6.25, 7.6])

ax[2].quiver(x_centroid, y_centroid, -cx_residual_reserved, cy_residual_reserved, angles='xy', color='black',
            scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0, pivot='middle')
ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='darkviolet')
ax[2].add_patch(circle2)
ax[2].invert_xaxis()
ax[2].set_xlabel('ra [deg]')
ax[2].set_title('Star - PSF ellipticity residuals')
ax[2].set_xlim([38.6, 37.2])
ax[2].set_ylim([6.25, 7.6])

ref_norm = 0.1  # Define reference vector norm
ref_x, ref_y = 38.4, 6.27  # Position to place reference vector
ax[0].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[1].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')
ax[2].quiver(ref_x, ref_y, ref_norm, 0, angles='xy', color='red',
             scale_units='xy', scale=scale, headlength=0, headwidth=0, headaxislength=0,
             label=f'Ref: {ref_norm}')

ax[0].text(ref_x + ref_norm / (2*scale), ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[1].text(ref_x + ref_norm / (2*scale), ref_y + 0.02, f'e=0.1', color='red', ha='center')
ax[2].text(ref_x + ref_norm / (2*scale), ref_y + 0.02, f'e=0.1', color='red', ha='center')

# ax[1].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[1].add_patch(circle1)
# ax[2].scatter([ra_bcg], [dec_bcg], marker='+', s=100, c='orange')
# ax[2].add_patch(circle1)
# 
fig.tight_layout()

## Maps of number of images in the coadd, e1, e2, and total ellipticity 

In [ ]:
e_psf_all = np.sqrt(e1_psf_all*e1_psf_all + e2_psf_all*e2_psf_all)

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,13))

scatter_plot1 = ax[0][0].scatter(merged_cat_all['coord_ra'],merged_cat_all['coord_dec'], c=merged_cat_all[f'{my_band}_inputCount'], s=1, cmap='viridis', marker='o')
ax[0][0].invert_xaxis()
ax[0][0].set_xlabel('ra [deg]')
ax[0][0].set_ylabel('dec [deg]')
plt.colorbar(scatter_plot1, ax=ax[0][0], label='number of images')

scatter_plot2 = ax[0][1].scatter(merged_cat_all['coord_ra'],merged_cat_all['coord_dec'], c=e_psf_all, s=1, cmap='viridis', marker='o')
ax[0][1].invert_xaxis()
ax[0][1].set_xlabel('ra [deg]')
ax[0][1].set_ylabel('dec [deg]')
plt.colorbar(scatter_plot2, ax=ax[0][1], label='ellipticity modulus')

scatter_plot3 = ax[1][0].scatter(merged_cat_all['coord_ra'],merged_cat_all['coord_dec'], c=e1_psf_all, s=1, cmap='viridis', marker='o')
ax[1][0].invert_xaxis()
ax[1][0].set_xlabel('ra [deg]')
ax[1][0].set_ylabel('dec [deg]')
plt.colorbar(scatter_plot2, ax=ax[1][0], label='e1')

scatter_plot4 = ax[1][1].scatter(merged_cat_all['coord_ra'],merged_cat_all['coord_dec'], c=e2_psf_all, s=1, cmap='viridis', marker='o')
ax[1][1].invert_xaxis()
ax[1][1].set_xlabel('ra [deg]')
ax[1][1].set_ylabel('dec [deg]')
plt.colorbar(scatter_plot2, ax=ax[1][1], label='e2')

fig.tight_layout()

## Histogram of the T, e1, e2 residuals in A360 field, for `used` and `reserved` stars

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

ax[0].hist((e1_star_used-e1_psf_used), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='used');
ax[0].hist((e1_star_reserved-e1_psf_reserved), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='reserved')
ax[0].set_xlabel(r'$\delta e_1 = $ Star e1 - PSF model e1')
ax[0].legend()

ax[1].hist((e2_star_used-e2_psf_used), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='used');
ax[1].hist((e2_star_reserved-e2_psf_reserved), bins=30, range=[-0.04, 0.04], density=True, alpha=0.2, label='reserved')
ax[1].set_xlabel(r'$\delta e_2 = $ Star e2 - model PSF e2')
ax[1].legend()

ax[2].hist((T_star_used-T_psf_used)/T_star_used, bins=30, range=[-0.1, 0.1], density=True, alpha=0.2, label='used');
ax[2].hist((T_star_reserved-T_psf_reserved)/T_star_reserved, bins=30, range=[-0.1, 0.1], density=True, alpha=0.2, label='reserved')
ax[2].set_xlabel(r'$\delta T / T_{star}$')
ax[2].legend()
fig.tight_layout()

## Radial profile of the tangential residuals (uses CLMM)

Another diagnostic test of the PSF correction consists in computing the tangential residuals from $\delta e_1$ and $\delta e_2$, and plot the corresponding radial profiles from the cluster BCG, for both `used` and `reserved` stars.

In [ ]:
import clmm
from clmm import GalaxyCluster, GCData, Cosmology
from clmm import Cosmology, utils

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

### Radial profile of the tangential residuals, for `used` and `reserved` stars

In [ ]:
from astropy.table import Table, vstack

In [ ]:
################ Reserved stars #############
galcat = GCData()
galcat['ra'] = merged_cat_reserved['coord_ra']
galcat['dec'] = merged_cat_reserved['coord_dec']
galcat['e1'] = e1_star_reserved - e1_psf_reserved # delta e1
galcat['e2'] = e2_star_reserved - e2_psf_reserved # delta e1

galcat['z'] = np.zeros(len(galcat['ra'])) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, 
                                coordinate_system='euclidean')

gc_object1.compute_tangential_and_cross_components(add=True);

#print(len(gc_object1.galcat))

#bins_mpc = clmm.make_bins(0.4,5,nbins=5, method='evenlog10width')
#bins_deg = clmm.make_bins(0.1,1,nbins=7, method='evenwidth')
bins_arcmin = clmm.make_bins(0,40,nbins=7, method='evenwidth')
gc_object1.make_radial_profile(bins=bins_arcmin, bin_units='arcmin', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=False, error_model='ste');

################ Used stars #############

galcat = GCData()
galcat['ra'] = merged_cat_used['coord_ra']
galcat['dec'] = merged_cat_used['coord_dec']
galcat['e1'] = e1_star_used - e1_psf_used
galcat['e2'] = e2_star_used - e2_psf_used
galcat['z'] = np.zeros(len(galcat['ra'])) # CLMM needs a redshift column for the source, even if not used

gc_object2 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, 
                                coordinate_system='euclidean')


gc_object2.compute_tangential_and_cross_components(add=True);

#print(len(gc_object2.galcat))

gc_object2.make_radial_profile(bins=bins_arcmin, bin_units='arcmin', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=False, error_model='ste');

#print(gc_object2.profile)

################ All stars #############
galcat_all = vstack([gc_object1.galcat, gc_object2.galcat])

gc_object3 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat_all, 
                                coordinate_system='euclidean')
gc_object3.compute_tangential_and_cross_components(add=True);

gc_object3.make_radial_profile(bins=bins_arcmin, bin_units='arcmin', add=True, cosmo=cosmo, overwrite=True, 
                               use_weights=False, error_model='ste');


print(f'Number of used stars = {len(gc_object2.galcat)}')
print(f'Number of reserved stars = {len(gc_object1.galcat)}')

plt.errorbar(gc_object2.profile['radius'], gc_object2.profile['gt'], gc_object2.profile['gt_err'], 
             ls='', marker='.', label="PSF stars - residuals")
plt.errorbar(gc_object1.profile['radius'], gc_object1.profile['gt'], gc_object1.profile['gt_err'], 
             ls='', marker='+', label="'reserved' stars - residuals")
plt.errorbar(gc_object3.profile['radius']+0.5, gc_object3.profile['gt'], gc_object3.profile['gt_err'], 
             ls='', marker='x', label=" all (PSF + reserved) stars - residuals")

#plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
plt.ylim([-0.008,0.008])
#plt.ylim([-0.06,0.05])
#plt.xlim([0.1,1])
plt.xlim([0.,40])
#plt.yscale('log')
#plt.xlabel('R [Mpc]')
plt.xlabel('Separation [arcmin]')
plt.ylabel(r'$\langle \delta e_t\rangle$')
plt.legend(loc=1)
plt.tight_layout()

## rho-statistics using AnalysisTools

In [ ]:
from lsst.analysis.tools.atools import RhoStatistics

In [ ]:
atool = RhoStatistics()

In [ ]:
atool.process.calculateActions.rho.treecorr.nbins = 21
atool.process.calculateActions.rho.treecorr.min_sep = 0.1
atool.process.calculateActions.rho.treecorr.max_sep = 100.0

In [ ]:
atool.finalize()

In [ ]:
input_schema = atool.getInputSchema()
needed_catalog_fields = [name[0] for name in list(atool.getInputSchema())]
print(needed_catalog_fields)

In [ ]:
prepResults = atool.prep(merged_cat_reserved, band=f"{my_band}")

In [ ]:
processResults = atool.process(prepResults, band=f"{my_band}")

In [ ]:
print("Mean angular separation:\n", processResults['rho1'].meanr, "\n")
print("Correlation function:\n", processResults['rho1'].xip, "\n")
print("Error in the correlation function:\n", processResults['rho1'].varxip)

In [ ]:
produceResults = atool.produce(processResults)#, band=f"{my_band}", skymap=skymap)


In [ ]:
rho_keys = ['rho1', 'rho2', 'rho3', 'rho4', 'rho5', 'rho3alt']
rho_titles = [fr'$\rho_1(\theta) = \langle\delta e, \delta e\rangle$',
             fr'$\rho_2(\theta) = \langle e, \delta e\rangle$',
             fr'$\rho_3(\theta) = \langle e \; \delta T/T , e \; \delta T/T\rangle$',
             fr'$\rho_4(\theta) = \langle \delta e, e \;\delta T/T\rangle$',
             fr'$\rho_5(\theta) = \langle e, e \;\delta T/T\rangle$',
             fr"$\rho_3\prime(\theta) = \langle\delta T/T, \delta T/T\rangle$"]

fig, axes = plt.subplots(2, 3, figsize=(12, 6), sharex=True, sharey=False)
axes = axes.flatten()

for i, key in enumerate(rho_keys):
    ax = axes[i]
    data = processResults[key]

    meanr = data.meanr
    yval = data.xi if key == 'rho3alt' else data.xip
    err = np.sqrt(data.varxi) if key == 'rho3alt' else np.sqrt(data.varxip)

    ax.errorbar(meanr, yval, yerr=err, fmt='o')
    ax.axhline(0, color='k', linestyle='--', linewidth=1)
    ax.set_xscale('log')
    ax.set_yscale('symlog', linthresh=1e-7)

#    ax.set_title(fr"$\rho_{{{key[-1]}}}(\theta)$")
#    ax.set_title(rho_titles[i])

    # Optional: add ±1e-6 shaded band
    ax.fill_between(meanr, 1e-6, -1e-6, color='gray', alpha=0.2)
    ax.set_ylabel(rho_titles[i])

# Axis labels
for ax in axes[3:]:
    ax.set_xlabel(r"$\theta$ [arcmin]")

fig.tight_layout()
#plt.show()